# Simulating InSAR Noise Data Using a Logairthmic Time Series

## 0. Importing Libraries

This notebook adjusts the dataset class created in the linear time series notebook for a logarithmic subsidence change.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset

#  1. Lambda Function for Linear/Nonlinear Subsidence Change


In our previous notebook we calculated the current pore pressure at a certain time as 

$\Delta P_t = -\Delta P_{\text{final}} \frac{t}{t_{\text{final}}} \$

The second part of this calculation, $\frac{t}{t_{\text{final}}}$, simply creates a linear time-series from 0 to the final time in our times array. If we tweak this formula to be a logarithimic change (or any non-linear time rate change) we are able to change the rate at which our subsidence changes.

Therefore we can rewrite our time series calculation as:

$\Delta P_t = -\Delta P_{\text{final}} f(x)$ 

Where the range of $f(t)$ is 0 to 1 inclusive. This is so that at the final time t, we get the max pore pressure change while at the first time t we get 0.

$0 \le f(t) \le 1$

For example to create a logarithmic change between 0 and 1 we need to use 

$f(t) = \frac{log(t) - log(t_{\text{0}})}{log(t_{\text{final}}) - log(t_{\text{0}})}$

We also need to change day 0 to be day 1 to avoid $log(0)$.

In [8]:
def get_times():
    return np.arange(1, 1461 + 1, 49)  

times = get_times()

f_x = [(np.log(t) - np.log(times[0])) / (np.log(times[-1]) - np.log(times[0])) for t in times]
f_x

[np.float64(0.0),
 np.float64(0.5388595330721226),
 np.float64(0.6329523454791063),
 np.float64(0.6883383530123444),
 np.float64(0.7277320942239894),
 np.float64(0.7583289711597873),
 np.float64(0.7833494028173682),
 np.float64(0.8045160859143178),
 np.float64(0.8228592351443302),
 np.float64(0.8390442475158096),
 np.float64(0.8535259084053924),
 np.float64(0.8666288526752489),
 np.float64(0.878592930125504),
 np.float64(0.8896003798945218),
 np.float64(0.8997929208733544),
 np.float64(0.9092829399377649),
 np.float64(0.9181610667388118),
 np.float64(0.9265014509117059),
 np.float64(0.934365530366754),
 np.float64(0.9418047805107296),
 np.float64(0.9488627582057442),
 np.float64(0.9555766470176288),
 np.float64(0.9619784430027674),
 np.float64(0.9680958769313561),
 np.float64(0.9739531402635706),
 np.float64(0.9795714629526987),
 np.float64(0.9849695779472404),
 np.float64(0.9901640980494473),
 np.float64(0.9951698242549775),
 np.float64(1.0)]

We can see that we can easily define $f(x)$ as a one line function based on the times array. Therefore, to simplify our dataset class we can implement an $f(x)$ lambda paramater into the class to make it easy to provide various time scales.

In [9]:
class Noise2NoiseDataset(Dataset):
    def __init__(self, size, S_max=5.0, D=50, nu=0.25, cm=1.0, V=1.0,
                 incidence_angle_deg=40, satellite_azimuth_deg=0,
                 random_noise_std=0.56, tropospheric_noise_beta=1.82, tropospheric_noise_scale=1.0,
                 total_days=1460, interval_days=49, f_t=-1):
        self.size = size
        self.S_max = S_max
        self.D = D
        self.nu = nu
        self.cm = cm
        self.V = V
        self.incidence_angle_deg = incidence_angle_deg
        self.satellite_azimuth_deg = satellite_azimuth_deg
        self.random_noise_std = random_noise_std
        self.tropospheric_noise_beta = tropospheric_noise_beta
        self.tropospheric_noise_scale = tropospheric_noise_scale
        self.total_days = total_days 
        self.interval_days = interval_days 

        self.times = self.get_times()
        self.total_time = self.times[-1] if len(self.times) > 1 else (self.times[0] if len(self.times) == 1 else 1.0)


    def __len__(self):
        return len(self.times)

    def generate_random_noise(self):
        return np.random.normal(loc=0.0, scale=self.random_noise_std, size=self.size)

    def generate_tropospheric_noise(self):
        noise = np.fft.fft2(np.random.randn(*self.size))

        ky = np.fft.fftfreq(self.size[0])
        kx = np.fft.fftfreq(self.size[1])

        kx, ky = np.meshgrid(kx, ky)

        k = np.sqrt(kx**2 + ky**2)
        k[0, 0] = 1e-7

        power = k ** (-self.tropospheric_noise_beta)

        frac_noise = np.fft.ifft2(noise * power).real

        frac_noise = (frac_noise - frac_noise.mean()) / frac_noise.std()
        return frac_noise * self.tropospheric_noise_scale

    @staticmethod
    def calculate_los_vector(incidence_angle_deg, satellite_azimuth_deg):
        incidence_angle_rad = np.deg2rad(incidence_angle_deg)
        satellite_azimuth_rad = np.deg2rad(satellite_azimuth_deg)
        look_azimuth_rad = satellite_azimuth_rad + np.pi/2

        l_east = np.sin(incidence_angle_rad) * np.sin(look_azimuth_rad)
        l_north = np.sin(incidence_angle_rad) * np.cos(look_azimuth_rad)
        l_up = np.cos(incidence_angle_rad)
        return np.array([l_east, l_north, l_up])

    def generate_subsidence(self, delta_P):
        y, x = np.indices(self.size)
        cx, cy = self.size[1] // 2, self.size[0] // 2
        r = np.sqrt((x - cx)**2 + (y - cy)**2)

        factor = (-1 / np.pi) * self.cm * (1 - self.nu) * delta_P * self.V
        uz = factor * (self.D / ((r**2 + self.D**2)**1.5))
        ur = factor * (r / ((r**2 + self.D**2)**1.5))

        azimuth = np.arctan2(y - cy, x - cx)

        ux = ur * np.cos(azimuth)
        uy = ur * np.sin(azimuth)

        los_vector = self.calculate_los_vector(self.incidence_angle_deg, self.satellite_azimuth_deg)

        simulated_interferogram = (ux * los_vector[0]) + (uy * los_vector[1]) + (uz * los_vector[2])

        return simulated_interferogram

    def get_times(self):
        return np.arange(1, self.total_days + 1, self.interval_days)

    def _get_clean_subsidence_image(self, t):
        delta_P_final = -self.S_max * ((np.pi * self.D**2) / (self.cm * (1 - self.nu) * self.V))
        if f_t == -1:
            delta_P_current = -delta_P_final * (t / self.total_time)
        else:
            delta_P_current = -delta_P_final * f_t(t, times)
        return self.generate_subsidence(delta_P=delta_P_current)

    def __getitem__(self, idx):
        current_time = self.times[idx]

        clean_image = self._get_clean_subsidence_image(current_time)

        noise1_random = self.generate_random_noise()
        noise1_tropo = self.generate_tropospheric_noise()
        noisy_image1 = clean_image + noise1_random + noise1_tropo

        noise2_random = self.generate_random_noise()
        noise2_tropo = self.generate_tropospheric_noise()
        noisy_image2 = clean_image + noise2_random + noise2_tropo

        noisy_image1_tensor = torch.from_numpy(noisy_image1).float().unsqueeze(0)
        noisy_image2_tensor = torch.from_numpy(noisy_image2).float().unsqueeze(0)

        return noisy_image1_tensor, noisy_image2_tensor

In [12]:
size = (1500, 1500)

f_t = lambda t, times: (np.log(t) - np.log(times[0])) / (np.log(times[-1]) - np.log(times[0]))
dataset = Noise2NoiseDataset(size=size, f_t = f_t)

# 2. Adding Orbit Types for Selecting Angles

Another thing we can add to mimic the realism of the satellite data is adding "Orbit Types." This would essentially replace our inputs of the incidence and azimuth angle and replace with just writing a word like "Ascending" or "Descending" and having preset values for these angles.

This gives us greater flexibility if we want to implement specific angles, orbit types, or scenarios when testing or training or model.

In [13]:
class Noise2NoiseDataset(Dataset):
    def __init__(self, size, S_max=5.0, D=50, nu=0.25, cm=1.0, V=1.0,
                 random_noise_std=0.56, tropospheric_noise_beta=1.82, tropospheric_noise_scale=1.0,
                 total_days=1460, interval_days=49, f_t=-1, orbit_type='ascending'):
        self.size = size
        self.S_max = S_max
        self.D = D
        self.nu = nu
        self.cm = cm
        self.V = V
        self.incidence_angle_deg = incidence_angle_deg
        self.satellite_azimuth_deg = satellite_azimuth_deg
        self.random_noise_std = random_noise_std
        self.tropospheric_noise_beta = tropospheric_noise_beta
        self.tropospheric_noise_scale = tropospheric_noise_scale
        self.total_days = total_days 
        self.interval_days = interval_days
        self.orbit_type = orbit_type
        
        self.incidence_angle_deg, self.satellite_azimuth_deg = self._get_orbit_geometry()
        self.times = self.get_times()
        self.total_time = self.times[-1] if len(self.times) > 1 else (self.times[0] if len(self.times) == 1 else 1.0)


    def __len__(self):
        return len(self.times)

    def generate_random_noise(self):
        return np.random.normal(loc=0.0, scale=self.random_noise_std, size=self.size)

    def generate_tropospheric_noise(self):
        noise = np.fft.fft2(np.random.randn(*self.size))

        ky = np.fft.fftfreq(self.size[0])
        kx = np.fft.fftfreq(self.size[1])

        kx, ky = np.meshgrid(kx, ky)

        k = np.sqrt(kx**2 + ky**2)
        k[0, 0] = 1e-7

        power = k ** (-self.tropospheric_noise_beta)

        frac_noise = np.fft.ifft2(noise * power).real

        frac_noise = (frac_noise - frac_noise.mean()) / frac_noise.std()
        return frac_noise * self.tropospheric_noise_scale

    @staticmethod
    def calculate_los_vector(incidence_angle_deg, satellite_azimuth_deg):
        incidence_angle_rad = np.deg2rad(incidence_angle_deg)
        satellite_azimuth_rad = np.deg2rad(satellite_azimuth_deg)
        look_azimuth_rad = satellite_azimuth_rad + np.pi/2

        l_east = np.sin(incidence_angle_rad) * np.sin(look_azimuth_rad)
        l_north = np.sin(incidence_angle_rad) * np.cos(look_azimuth_rad)
        l_up = np.cos(incidence_angle_rad)
        return np.array([l_east, l_north, l_up])

    def generate_subsidence(self, delta_P):
        y, x = np.indices(self.size)
        cx, cy = self.size[1] // 2, self.size[0] // 2
        r = np.sqrt((x - cx)**2 + (y - cy)**2)

        factor = (-1 / np.pi) * self.cm * (1 - self.nu) * delta_P * self.V
        uz = factor * (self.D / ((r**2 + self.D**2)**1.5))
        ur = factor * (r / ((r**2 + self.D**2)**1.5))

        azimuth = np.arctan2(y - cy, x - cx)

        ux = ur * np.cos(azimuth)
        uy = ur * np.sin(azimuth)

        los_vector = self.calculate_los_vector(self.incidence_angle_deg, self.satellite_azimuth_deg)

        simulated_interferogram = (ux * los_vector[0]) + (uy * los_vector[1]) + (uz * los_vector[2])

        return simulated_interferogram

    def get_times(self):
        return np.arange(1, self.total_days + 1, self.interval_days)

    def _get_clean_subsidence_image(self, t):
        delta_P_final = -self.S_max * ((np.pi * self.D**2) / (self.cm * (1 - self.nu) * self.V))
        if f_t == -1:
            delta_P_current = -delta_P_final * (t / self.total_time)
        else:
            delta_P_current = -delta_P_final * f_t(t, times)
        return self.generate_subsidence(delta_P=delta_P_current)

    def _get_orbit_geometry(self):
        if self.orbit_type == 'ascending':
            return 40, 15
        elif self.orbit_type == 'descending':
            return 40, 195
        else:
            raise ValueError("Invalid orbit type")

    def __getitem__(self, idx):
        current_time = self.times[idx]

        clean_image = self._get_clean_subsidence_image(current_time)

        noise1_random = self.generate_random_noise()
        noise1_tropo = self.generate_tropospheric_noise()
        noisy_image1 = clean_image + noise1_random + noise1_tropo

        noise2_random = self.generate_random_noise()
        noise2_tropo = self.generate_tropospheric_noise()
        noisy_image2 = clean_image + noise2_random + noise2_tropo

        noisy_image1_tensor = torch.from_numpy(noisy_image1).float().unsqueeze(0)
        noisy_image2_tensor = torch.from_numpy(noisy_image2).float().unsqueeze(0)

        return noisy_image1_tensor, noisy_image2_tensor

# 3. Metrics for Comparing Clean and Denoised Outputs

The original linear notebook leaves out metrics for comparing our images to evaluate our model with. There are three primary metrics often used for denoising images this

In [4]:
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric

def generate_metrics(clean_image, denoised_image):
    clean_image_np = clean_image.squeeze().cpu().numpy()
    denoised_output_np = denoised_output.squeeze().cpu().numpy()
    
    max_val = np.max(clean_image)
    min_val = np.min(clean_image)
    data_range = max_val - min_val

    psnr_metric(clean_image, denoised_image, data_range=data_range)
    ssim_value = ssim_metric(clean_image, denoised_image, data_range=data_range)
    rsme_value = np.sqrt(np.mean((clean_image - denoised_image)**2))

    print(f"PSNR: {psnr_value:.4f} dB")
    print(f"SSIM: {ssim_value:.4f}")
    print(f"RMSE: {rmse_value:.4f} (cm)")